In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 

In [2]:
traindf = pd.read_hdf("../data/traindf_clean.hdf")
testdf = pd.read_hdf("../data/test_clean.hdf")

In [3]:
print(list(traindf.columns))
print(list(testdf.columns))

traindf = traindf.sort_values(by=['srch_id'])

['prop_id', 'srch_id', 'position', 'price_quality', 'competitor_lower', 'competitor_available', 'visited_before', 'click_bool', 'booking_bool', 'price_usd', 'promotion_flag', 'prop_brand_bool', 'random_bool', 'prop_location_score1', 'prop_location_score2', 'prop_id_price_mean', 'avg_price_propid', 'std_avg_price_propid', 'amount_hotels', 'avg_price_propid_after', 'std_avg_price_propid_after', 'importance']
['prop_id', 'srch_id', 'price_quality', 'competitor_lower', 'competitor_available', 'visited_before', 'PriceBand', 'price_usd', 'promotion_flag', 'prop_brand_bool', 'random_bool', 'prop_location_score1', 'prop_location_score2', 'prop_id_price_mean']


In [4]:
# split label and other variables
x_train, y_train = traindf, traindf["importance"]
positions = pd.DataFrame(y_train).set_index(traindf.srch_id)

# 1 procent van totale data
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42, shuffle=False, stratify = None)
x_traincopy = x_train.copy()
x_train = x_train.drop(columns=["position", "importance", "competitor_available",
                               "competitor_lower", "visited_before"])

# x_test, y_test = testdf.drop(columns=["position", "click_bool", "booking_bool"]), testdf["position"]
x_test = testdf

x_train.shape, y_train.shape, x_test.shape

((3966677, 17), (3966677,), (4959183, 14))

In [5]:
params = {'objective': 'rank:pairwise', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6,  'n_estimators': 500}

# groups equal to length of queries
query_lengths = x_train.groupby('srch_id').size().values
    

In [6]:
model = xgb.sklearn.XGBRanker(**params)
model.fit(x_train, y_train, query_lengths, verbose=True)

KeyboardInterrupt: 

In [ ]:
# Make fake column of positions to test score in ndcg scoring function
x_train["position_temp"] = x_train.groupby(['srch_id']).cumcount()+1

In [ ]:
# x_train["position"] = y_train
predictions_sorted = []
prop_ids_sorted = []

# x_train = x_train.drop(["position", "pred"], axis=1)

# Sort predictions for each group SEPERATELY
for srchid, group in x_train.groupby('srch_id'):
    
    # Predictions for one search_id
    pred = model.predict(group.drop("position_temp", axis=1))
    
    # Sort all columns (position and prop_id) from this group based on predictions
    predictions = [x for _,x in sorted(zip(pred, group.position_temp), reverse=True)] # TODO: Reverse = False toch? Hoezo werkt dit..
    predictions_sorted.append(predictions)
    
    prop_id = [x for _,x in sorted(zip(pred, group.prop_id), reverse=True)]
    prop_ids_sorted.append(prop_id)
    
# Flatten lists
predictions_sorted = [item for sublist in predictions_sorted for item in sublist]
prop_ids_sorted = [item for sublist in prop_ids_sorted for item in sublist]


In [ ]:
x_traincopy["prop_id"] = prop_ids_sorted
x_traincopy["predicted_rank"] = predictions_sorted
final_df = x_traincopy
final_df.head()

In [ ]:
final_df["predicted_rank"] = final_df["position_temp"]
final_df.head()

In [ ]:
# pred = model.predict(x_train.drop("position", axis=1))
# x_traincopy["pred"] = pred
# x_temp = x_traincopy.sort_values(by=['pred'], ascending=False)["position"]

# predictions_sorted = []
# x_train["position"] = y_train

# # Sort predictions for each group SEPERATELY
# for srchid, group in x_train.groupby('srch_id'):
    
#     # Predictions for one search_id
#     pred = model.predict(group.drop("position", axis=1))
# #     print(pred, group)
# #     group['predicted_rank'] = pred
# #     group = group.sort_values('predicted_rank')
        
#     predictions = [x for _,x in sorted(zip(pred, group.position), reverse=True)] # TODO: Reverse = False toch? Hoezo werkt dit..
#     predictions_sorted.append(predictions)
    
# predictions_sorted = [item for sublist in predictions_sorted for item in sublist]




In [ ]:
x_traincopy["predicted_rank"] = x_temp

In [ ]:
x_traincopy.to_pickle("../pickles/XGBoost_solution_test8mei.pkl")

## Test set only

In [ ]:
# pred = model.predict(x_test)

# x_test["predicted_rank"] = pred
# predictions_sorted = []

# # Sort predictions for each group SEPERATELY
# for srchid, group in x_test.groupby('srch_id'):
    
#     predictions = [x for _,x in sorted(zip(group.predicted_rank, group.prop_id))]
#     predictions_sorted.append(predictions)
    
# predictions_sorted = [item for sublist in predictions_sorted for item in sublist]

In [ ]:
# final_df = x_test[["srch_id", "prop_id"]]
# final_df.to_csv("submission5mei.csv", columns=["srch_id", "prop_id"], index=False)

In [ ]:
# # Test if it worked
# test = pd.read_csv("submission5mei.csv")
# test.head()

In [ ]:
# importances = pd.DataFrame(x_test.columns, model.feature_importances_)
# importances.head(10)

In [ ]:
x_traincopy.head(5)